# SV Simulation Tutorial:
This notebook demonstrates the process of creating a synthetic genome from an input reference, and then simulating and aligning reads and inspecting the results. Below we provide an example configuration file specifying the SVs to be included in the synthetic output genome and insert those SVs into chromosome 21 of GRCh38. We provide example calls to DWGSIM and PBSIM3 to generate synthetic paired-end short read and HiFi long reads, but this procedure is generalizable to any read simulator. Lastly, we provide visualization code to view the size distribution of the simulated SVs as well as the pileup images of the reads in the impacted regions of the genome.

In [ ]:
import igv_notebook
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from pysam import VariantFile
from intervaltree import Interval, IntervalTree
from IPython.display import Image

## Synthetic reference simulation
`config.yaml` gives the composition of the set of SVs to be input into the reference. Here we've included 5 examples each of the types deletion (DEL), tandem duplication (DUP), inversion (INV), and dispersed duplication (dDUP).

In [ ]:
%%sh
cat ./config.yaml

Creating output directory and downloading chr21 reference

In [ ]:
%%sh
mkdir -p output
mkdir -p chr21_ref

wget --directory-prefix=chr21_ref/ https://hgdownload.soe.ucsc.edu/goldenPath/hg38/chromosomes/chr21.fa.gz
gunzip chr21_ref/chr21.fa

samtools faidx chr21_ref/chr21.fa

`simulate.py` is called with the above config and the chromosome 21 reference as input, specifies `./output/sim` as the output prefix for the files that will be generated, and sets a random seed for the simulation (an optional input for `simulate.py`).

In [ ]:
%%sh
python ../simulate.py chr21_ref/chr21.fa ./config.yaml ./output/sim --random_seed 42

Below we show the VCF generated from the simulation

In [ ]:
%%sh
cat output/sim.vcf

## Read simulation
### Short-read simulation
The command below is set to generate synthetic paired-end short reads at 10x coverage (5x per haplotype). Definitions for some of the DWGSIM input parameters are given below:
```
         -C FLOAT      mean coverage across available positions (-1 to disable) [100.00]
         -1 INT        length of the first read [70]
         -2 INT        length of the second read [70]
         -y FLOAT      probability of a random DNA read [0.05]
         -S INT        generate reads [0]:
                           0: default (opposite strand for Illumina, same strand for SOLiD/Ion Torrent)
                           1: same strand (mate pair)
                           2: opposite strand (paired end)
```

In [ ]:
%%sh
DWGSIM_PATH=dwgsim
$DWGSIM_PATH -C 5 -1 151 -2 151 -y 0 -S 0 -c 0 -m /dev/null -H output/sim.hapA.fa output/sim_sr.dwgsim.hap.1
$DWGSIM_PATH -C 5 -1 151 -2 151 -y 0 -S 0 -c 0 -m /dev/null -H output/sim.hapB.fa output/sim_sr.dwgsim.hap.2

After generating the haplotype-specific read sets, we combine them and align with BWA

In [ ]:
%%sh
mv output/sim_sr.dwgsim.hap.1.bfast.fastq.gz output/sim_sr.dwgsim.hap.12.fastq.gz
cat output/sim_sr.dwgsim.hap.2.bfast.fastq.gz >> output/sim_sr.dwgsim.hap.12.fastq.gz
rm output/sim_sr.dwgsim.hap.2.bfast.fastq.gz

In [ ]:
%%sh
BWA_PATH=../../bwa-0.7.17/bwa
$BWA_PATH index chr21_ref/chr21.fa
$BWA_PATH mem -p chr21_ref/chr21.fa output/sim_sr.dwgsim.hap.12.fastq.gz | samtools view -Sb - > output/sim_sr.bwamem.bam

After alignment, we sort and index the .bam

In [ ]:
%%sh
samtools sort output/sim_sr.bwamem.bam > output/sim_sr.bwamem.sorted.bam

In [ ]:
%%sh
rm output/sim_sr.bwamem.bam
samtools index output/sim_sr.bwamem.sorted.bam

### Long-read simulation
The commands below use PBSIM3 to simulate HiFi reads from the synthetic genome (again at 10x total coverage). As with the short read simulation above, after generating the reads we combine the two haplotypes' reads and and create a .bam (with minimap2 in this case) which we then sort and index. 

In [ ]:
%%sh
PBSIM_PATH=../../pbsim3/src/pbsim
PBSIM_MODEL_PATH="$(dirname "$(dirname "$PBSIM_PATH")")"/data/QSHMM-RSII.model
$PBSIM_PATH --strategy wgs --method qshmm --qshmm $PBSIM_MODEL_PATH --depth 5 --accuracy-mean 0.999 --accuracy-min 0.99 --length-min 18000 --length-mean 20000 --length-max 22000 --genome output/sim.hapA.fa --prefix output/sim_lr.hapA
$PBSIM_PATH --strategy wgs --method qshmm --qshmm $PBSIM_MODEL_PATH --depth 5 --accuracy-mean 0.999 --accuracy-min 0.99 --length-min 18000 --length-mean 20000 --length-max 22000 --genome output/sim.hapB.fa --prefix output/sim_lr.hapB

In [ ]:
%%sh
mv output/sim_lr.hapA_0001.fastq output/sim_lr.hap12.fastq
cat output/sim_lr.hapB_0001.fastq >> output/sim_lr.hap12.fastq
rm output/sim_lr.hapB_0001.fastq
rm output/*_0001.maf
rm output/*_0001.ref

In [ ]:
%%sh
MINIMAP_PATH=../../minimap2-2.26/minimap2
$MINIMAP_PATH --eqx -ax map-hifi chr21_ref/chr21.fa output/sim_lr.hap12.fastq | samtools view -Sb > output/sim_lr.minimap2.bam

In [ ]:
%%sh
samtools sort output/sim_lr.minimap2.bam > output/sim_lr.minimap2.sorted.bam

In [ ]:
%%sh
rm output/sim_lr.minimap2.bam
samtools index output/sim_lr.minimap2.sorted.bam

------

## SV Size Visualization
Below we provide a simple parsing function to convert the VCF generated above into a dataframe that can easily be plotted or fed to other pos-hoc analyses.

In [ ]:
def insilico_bench_to_df(input_vcf):
    vcf = VariantFile(input_vcf)
    callset_info = {'chrom': [], 'start': [], 'end': [], 'component': [], 'length': [], 'type': [], 'parent_type': [], 'context': []}
    for rec in vcf.fetch():
        callset_info['chrom'].append(rec.chrom)
        callset_info['start'].append(rec.start)
        callset_info['end'].append(rec.stop)
        callset_info['parent_type'].append(rec.id)
        callset_info['type'].append(rec.id)
        callset_info['component'].append('source')
        callset_info['length'].append(rec.info['SVLEN'])
        callset_info['context'].append('None' if 'OVERLAP_EV' not in rec.info else rec.info['OVERLAP_EV'])
        if 'TARGET' in rec.info:
            disp_interval = (rec.stop, rec.info['TARGET']) if rec.info['TARGET'] > rec.stop else (rec.info['TARGET'], rec.start)
            callset_info['chrom'].append(rec.chrom)
            callset_info['start'].append(rec.start)
            callset_info['end'].append(rec.stop)
            callset_info['parent_type'].append(rec.id)
            callset_info['type'].append(rec.id + '_disp')
            callset_info['component'].append('dispersion')
            callset_info['length'].append(disp_interval[1] - disp_interval[0])
            callset_info['context'].append('None')
    vcf_df = pd.DataFrame(callset_info)
    return vcf_df


In [ ]:
sim_df = insilico_bench_to_df('output/sim.vcf')

In [ ]:
sim_df

In [ ]:
def plot_simulation(df):
    violin_color_palette = {'source': '#85C1E9', 'dispersion': '#EB984E'}
    f, ax = plt.subplots(figsize=(10,5))
    sns.set_style('white')
    sns.set_style('ticks')
    sns.violinplot(data=df, x='parent_type', y='length', hue='component', split=False, order=['DEL', 'DUP', 'INV', 'dDUP'], hue_order=['source', 'dispersion'], palette=violin_color_palette)
    sns.despine(offset=10, trim=True)
    ax.set(xlabel='', ylabel='Length')

In [ ]:
plot_simulation(sim_df)

## IGV Visualization
Below we provide infrastructure for the automatic visualization of insilicoSV output in IGV. The IGV batch script generated by this function can be input into desktop or commandline IGV, but for this example we will call into commandline IGV, which can be downloaded [here](https://data.broadinstitute.org/igv/projects/downloads/2.16/IGV_2.16.2.zip).

In [ ]:
%%sh
mkdir -p IGV_screenshots

In [ ]:
def is_contained(query_interval, interval):
    # helper function to check for containment of query interval in any of the intervals stored in the tree
    return interval.begin <= query_interval.begin and\
           interval.end >= query_interval.end and\
           interval.data == query_interval.data


def generate_script(input_vcfs, bam_paths, vcf_paths, output_batch_script_path, igv_screenshot_dir, min_svlen,
                    max_svlen, genome='hg38', colorby_ins_size=True, groupby_pair_orientation=True,
                    viewaspairs=True, skip_duplicates=True):
    """
    method to generate a batch script for an input vcf
    """
    out = open(output_batch_script_path, 'w')
    out.write('new\n')
    out.write(f'genome {genome}\n')
    out.write('preference SAM.MAX_VISIBLE_RANGE 1000\n')
    out.write('preference SAM.SHOW_MISMATCHES FALSE\n')
    for bam_path in bam_paths:
        out.write(f'load {bam_path}\n')
    for vcf_path in vcf_paths:
        out.write(f'load {vcf_path}\n')

    # maintaining an interval tree of the genome intervals that have been screenshot in case
    # there are multiple input records that will have been captured by a single screenshot
    screenshot_intervals = IntervalTree()

    write_first_time = True
    for input_vcf in input_vcfs:
        input_vcf_file = VariantFile(input_vcf)
        for rec in input_vcf_file.fetch():
            if skip_duplicates and any([is_contained(Interval(rec.start, rec.stop, rec.chrom), ivl)
                                        for ivl in list(screenshot_intervals.overlap(rec.start, rec.stop))]):
                continue
            start, end = rec.start, rec.stop
            if 'TARGET' in rec.info:
                if rec.info['TARGET'] > end:
                    end = rec.info['TARGET']
                else:
                    start = rec.info['TARGET']
            svlen = end - start
            if min_svlen <= svlen <= max_svlen:
                # setting the margin on either side of the interval to svlen/10
                screenshot_margin = svlen // 10
                start_pos = str(start - screenshot_margin)
                end_pos = str(end + screenshot_margin)
                svtype = rec.info['SVTYPE']
                out.write('goto ' + rec.chrom + ':' + start_pos + '-' + end_pos + '\n')
                screenshot_intervals[int(start_pos):int(end_pos)] = rec.chrom
                if write_first_time:
                    if colorby_ins_size:
                        out.write('colorby INSERT_SIZE\n')
                    if groupby_pair_orientation:
                        out.write('group PAIR_ORIENTATION\n')
                    if viewaspairs:
                        out.write('viewaspairs\n')
                    out.write('maxPanelHeight 1000\n')
                    out.write('snapshotDirectory ' + igv_screenshot_dir + '\n')
                    write_first_time = False
                out.write('collapse\n')
                out.write(f'snapshot {rec.chrom}_{rec.start}_{rec.stop}_{svtype}.png\n')
    out.close()

generate_script(input_vcfs=['output/sim.vcf'], bam_paths=['output/sim_sr.bwamem.sorted.bam', 'output/sim_lr.minimap2.sorted.bam'],
                vcf_paths=['output/sim.vcf'], output_batch_script_path='IGV_screenshots/IGV_batch_script.txt',
                igv_screenshot_dir='./IGV_screenshots', min_svlen=0, max_svlen=250000)

The batch script is populated with some initializing parameters regarding alignment visualization but can be edited according to the [documentation](https://github.com/igvteam/igv/wiki/Batch-commands)

In [ ]:
%%sh
cat IGV_screenshots/IGV_batch_script.txt | head -n 20

This call to IGV will populate the `IGV_screenshots/` directory with `.png` images for each record in our input VCF, a subset of which we visualize below.

In [ ]:
%%sh
PATH_TO_IGV=~/Documents/software/IGV_2.16.0/igv.sh
sh $PATH_TO_IGV -b IGV_screenshots/IGV_batch_script.txt

In [ ]:
Image(filename='IGV_screenshots/chr21_40878089_40878743_DEL.png')

In [ ]:
Image(filename='IGV_screenshots/chr21_26921713_26922366_DUP.png')

In [ ]:
Image(filename='IGV_screenshots/chr21_9370432_9370586_INV.png')

In [ ]:
Image(filename='IGV_screenshots/chr21_9285293_9285424_dDUP.png')